# 국어 - 출제의도 찾아서 자르기

In [ ]:
import cv2
import easyocr
import re
import glob
import os

# 이미지 파일들이 있는 디렉토리 경로
directory = r'C:\Users\hana9\preprocessing\KOR_lr\2015_G1_11_kor_exp\cropped'

# 결과 이미지를 저장할 디렉토리 경로
output_directory = r'C:\Users\hana9\preprocessing\KOR_lr\2015_G1_11_kor_exp\cropped'

# 해당 디렉토리 내의 모든 png 파일 찾기
png_files = glob.glob(os.path.join(directory, '*.png'))

def extract_sort_key(filename):
    # 파일명에서 left 또는 right 부분과 숫자 부분을 추출하여 정렬 키를 반환하는 함수
    match = re.match(r'.*_(left|right)__page(\d+)\.png', filename)
    if match:
        prefix = match.group(1)
        number = int(match.group(2))
        return (number, prefix)  # 숫자를 먼저 비교하고, 그 다음에 left/right를 비교하여 정렬

    # 매칭되지 않는 경우는 마지막으로 처리
    return (float('inf'), filename)

# 파일을 정렬합니다.
sorted_files = sorted(png_files, key=extract_sort_key)

# EasyOCR을 사용하여 텍스트 인식
reader = easyocr.Reader(['ko', 'en'])

# 정규 표현식으로 매칭되는 패턴들 정의
pattern1 = re.compile(r"\b([0-9]|[1-3][0-9]|4[0-5])\.\s*")
pattern2 = re.compile(r'^(?:\d{1,2}\s*<|\d{1,2}\s*윗글)')
pattern3 = re.compile(r'\[(\d+)\s*~\s*(\d+)\]|\[\d+\s*\b')

patterns = [pattern1, pattern2, pattern3]

# 각 이미지에 대해 처리
for idx, f_name in enumerate(sorted_files):
    # 이미지 파일 읽기
    img = cv2.imread(f_name)

    # EasyOCR을 사용하여 텍스트 인식
    results = reader.readtext(f_name)

    # 정규 표현식으로 매칭되는 패턴 찾기
    def find_patterns(results, patterns):
        found_patterns = []
        for pattern in patterns:
            for (bbox, text, prob) in results:
                if re.match(pattern, text):
                    found_patterns.append((bbox, text, prob))
        return found_patterns

    # 패턴 찾기
    found_patterns = find_patterns(results, patterns)

    if found_patterns:
        # 패턴 출력 (디버깅용)
        print("Found patterns:", found_patterns)

        # y 좌표로 정렬
        found_patterns.sort(key=lambda b: b[0][0][1])

        # 정렬된 결과 출력
        print("Sorted patterns by y-coordinate:")
        for pattern in found_patterns:
            print(pattern)

        # 첫 번째 패턴의 상단 y 좌표 구하기
        first_y_min = int(found_patterns[0][0][0][1])

        # 첫 번째 패턴이 y=50보다 큰 경우, 원본 이미지의 상단부터 첫 번째 패턴까지 자르기
        if first_y_min > 50:
            cropped_img = img[0:first_y_min:, :]

            # 결과 이미지 저장
            output_file = os.path.join(output_directory, f"{idx}_top_to_first.png")
            cv2.imwrite(output_file, cropped_img)

        # 패턴이 있는 텍스트의 내용과 위치 정보를 활용하여 처리
        for i, (bbox1, text1, _) in enumerate(found_patterns):
            if i < len(found_patterns) - 1:
                bbox2, text2, _ = found_patterns[i + 1]

                # 첫 번째 패턴의 상단과 두 번째 패턴의 상단 사이의 이미지 부분 자르기
                x_min = min(int(bbox1[0][0]), int(bbox2[0][0]))
                y_min = min(int(bbox1[0][1]), int(bbox2[0][1]))
                x_max = max(int(bbox1[2][0]), int(bbox2[2][0]))
                y_max = max(int(bbox1[0][1]), int(bbox2[0][1]))

                cropped_img = img[y_min:y_max, x_min:]

            # 결과 이미지 저장
            # output_file = os.path.join(output_directory, f"{idx}_{i}.png")
            # cv2.imwrite(output_file, cropped_img)

                # 결과 이미지 저장 (found_patterns에서 숫자 추출하여 파일명으로 사용)
                pattern_number = re.findall(r'\d+', text1)[0]  # found_patterns에서 숫자 추출
                output_file = os.path.join(output_directory, f"{idx}_{pattern_number}_{i}.png")
                cv2.imwrite(output_file, cropped_img)

        # 마지막 패턴의 상단부터 원본 이미지의 끝까지 자르기
        last_bbox, last_text, _ = found_patterns[-1]
        x_min, y_min = int(last_bbox[0][0]), int(last_bbox[0][1])
        # y_min = int(last_bbox[0][1])
        cropped_img = img[y_min:-320, x_min:]

        # 결과 이미지 저장 (found_patterns에서 숫자 추출하여 파일명으로 사용)
        pattern_number = re.findall(r'\d+', text1)[0]  # found_patterns에서 숫자 추출
        output_file = os.path.join(output_directory, f"{idx}_{pattern_number}_{i}.png")
        cv2.imwrite(output_file, cropped_img)

    else:
        # 패턴이 발견되지 않으면 전체 이미지를 저장
        height, width, _ = img.shape
        cropped_img = img[0:height, 0:width]

        # 결과 이미지 저장 (파일명을 임의로 지정)
        output_file = os.path.join(output_directory, f"whole_image_{idx}.png")
        cv2.imwrite(output_file, cropped_img)

In [1]:
import os
import cv2
import numpy as np
from pdf2image import convert_from_path
from PIL import Image
import easyocr
import re

class ImageProcessor_kor:
    def __init__(self, image_path):
        self.image_path = image_path
        self.header = None
        self.body = None
        self.left_part = None
        self.middle_part = None
        self.right_part = None

    def process_image(self):
        src = cv2.imread(self.image_path)
        gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        edges = cv2.Canny(blurred, 50, 150, apertureSize=3)

        # 이진화
        _, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY_INV)

         # 윤곽선 찾기
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # 가로선 찾기
        horizontal_lines = []
        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            if w > 700 and h < 50:  # 너비가 특정 값 이상이고 높이가 작은 가로선 찾기
                horizontal_lines.append((x, y, w, h))

        # 제일 작은 y값을 가진 가로선 찾기
        if horizontal_lines:
            horizontal_lines.sort(key=lambda line: line[1])  # y 값을 기준으로 정렬
            smallest_y_line = horizontal_lines[0]  # y 값이 제일 작은 가로선
            x, y, w, h = smallest_y_line
            cut_line = y + h // 2  # 가로선의 중심 위치 계산

            # 상단과 하단 부분 분리
            header = src[:cut_line, :]
            body = src[cut_line:, :]

            self.header = header
            self.body = body

            height, width, _ = self.body.shape
            # 가로로 3등분 하기 위한 위치 계산
            cut_width = width // 3

            # 좌측, 중간, 우측 부분 분리
            self.left_part = src[:, :cut_width-5]
            self.middle_part = src[:, cut_width:cut_width*2]
            self.right_part = src[:, (cut_width*2)+5:]

    def save_processed_images(self, output_path_left, output_path_middle, output_path_right):
        if self.left_part is None or self.middle_part is None or self.right_part is None:
            print("이미지가 처리되지 않았습니다. process_image() 메서드를 호출하여 이미지를 처리하세요.")
            return
        cv2.imwrite(output_path_left, self.left_part)
        cv2.imwrite(output_path_middle, self.middle_part)
        cv2.imwrite(output_path_right, self.right_part)

# 파일 경로 설정
image_path = r"C:\Users\hana9\preprocessing\KOR_lr\2015_G1_11_kor_exp\kor_page0.png"
output_path_left = r"C:\Users\hana9\preprocessing\KOR_lr\2015_G1_11_kor_exp\cropped/kor_page1_left.png"
output_path_middle = r"C:\Users\hana9\preprocessing\KOR_lr\2015_G1_11_kor_exp\cropped/kor_page1_middle.png"
output_path_right = r"C:\Users\hana9\preprocessing\KOR_lr\2015_G1_11_kor_exp\cropped/kor_page1_right.png"

# 이미지 처리 및 저장
processor = ImageProcessor_kor(image_path)
processor.process_image()
processor.save_processed_images(output_path_left, output_path_middle, output_path_right)


이미지가 처리되지 않았습니다. process_image() 메서드를 호출하여 이미지를 처리하세요.


# 전처리

In [11]:
# Import required libraries
import os
import cv2
import numpy as np
from pdf2image import convert_from_path
from PIL import Image
import easyocr
import re

class ImageProcessor_kor:
    def __init__(self, image_path):
        self.image_path = image_path
        self.processed_image = None
        self.left_part = None
        self.middle_part = None
        self.right_part = None

    def process_image(self):
        src = cv2.imread(self.image_path)
        gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
        
        # 텍스트 '정답 및 해설'의 영역 찾기
        text = "정답 및 해설"
        reader = easyocr.Reader(['ko'])
        result = reader.readtext(src)
        
        header_y = 0
        for (bbox, text, prob) in result:
            if "정답 및 해설" in text:
                header_y = bbox[2][1]  # bbox의 하단 y 좌표
                break
        
        # 상단 '정답 및 해설' 부분 자르기
        src = src[int(header_y):, :]
        
        self.processed_image = src

        height, width, _ = src.shape
        cut_width = width // 3
        
        left_margin = 80  # 좌측 여백
        right_margin = 10  # 우측 여백
        
        # 좌측, 중간, 우측 부분 자르기
        self.left_part = src[:, :cut_width + right_margin]
        self.middle_part = src[:, cut_width:cut_width*2]
        self.right_part = src[:, cut_width*2 - left_margin:]

    def save_processed_images(self, output_path_left, output_path_middle, output_path_right):
        if self.processed_image is None:
            print("이미지가 처리되지 않았습니다. process_image() 메서드를 호출하여 이미지를 처리하세요.")
            return
        cv2.imwrite(output_path_left, self.left_part)
        cv2.imwrite(output_path_middle, self.middle_part)
        cv2.imwrite(output_path_right, self.right_part)

# 파일 경로 설정
image_path = r"C:\Users\hana9\preprocessing\KOR_lr\2015_G1_11_kor_exp\kor_page0.png"
output_path_left = r"C:\Users\hana9\preprocessing\KOR_lr\2015_G1_11_kor_exp\cropped/kor_page1_left.png"
output_path_middle = r"C:\Users\hana9\preprocessing\KOR_lr\2015_G1_11_kor_exp\cropped/kor_page1_middle.png"
output_path_right = r"C:\Users\hana9\preprocessing\KOR_lr\2015_G1_11_kor_exp\cropped/kor_page1_right.png"

# 이미지 처리 및 저장
processor = ImageProcessor_kor(image_path)
processor.process_image()
processor.save_processed_images(output_path_left, output_path_middle, output_path_right)


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
